## Make patch list excel
Read image list and convert it to excel

In [61]:
import os
import pandas as pd
from natsort import natsorted

img_home = 'KOMPSAT-5/Images_patch'
sheet_name = 'KOMPSAT-5'
excel_path = 'SAR_patches.xlsx'

patch_list = natsorted(os.listdir(img_home))
scene_list = [name[:50] for name in patch_list]
df = pd.DataFrame({'No.': list(range(1, len(patch_list)+1)),
                   'Scene': scene_list,
                   'Patch': patch_list})
df['Worker'] = ''  # add an empty column
df['Done'] = ''

# Get the dimensions of the dataframe.
(max_row, max_col) = df.shape

# Create the list where we 'll capture the cells that appear for 1st time,
# add the 1st row and we start checking from 2nd row until end of df
startCells = [1]
for row in range(2,len(df)+1):
    if (df.loc[row-1,'Scene'] != df.loc[row-2,'Scene']): 
        startCells.append(row)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(excel_path, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# header and index and skip one row to allow us to insert a user defined header.
df.to_excel(writer, sheet_name=sheet_name, startrow=0, header=True, index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets[sheet_name]

for row in startCells:
    try:
        endRow = startCells[startCells.index(row)+1]-1
        if row != endRow:
            worksheet.merge_range(row, 1, endRow, 1, df.loc[row-1,'Scene'])
    except IndexError as e:
        if row != max_row:
            worksheet.merge_range(row, 1, max_row, 1, df.loc[row-1,'Scene'])
        else:
            print(e)

# Set formats of columns and head 
head_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1, 'bg_color': 'D9D9D9'})
border_format = workbook.add_format({'border': 1})
center_format = workbook.add_format({'align': 'center', 'valign': 'vcenter'})
left_format = workbook.add_format({'align': 'left', 'valign': 'vcenter'})

worksheet.conditional_format(1, 0, max_row, max_col-1, {'type': 'no_blanks', 'format': border_format})
worksheet.conditional_format(1, 0, max_row, max_col-1, {'type': 'blanks', 'format': border_format})
worksheet.conditional_format(1, 2, max_row, 2, {'type': 'no_blanks', 'format': border_format})
worksheet.conditional_format(0, 0, 0, max_col-1, {'type': 'no_blanks', 'format': head_format})
worksheet.set_column(0,max_col-1, None, center_format)
worksheet.set_column(2,2, None, left_format)
worksheet.autofit()  # autofitting of column widths
            
# Close the Pandas Excel writer and output the Excel file.
writer.close()